# 项目简介

## 业务背景

下图是火花课堂的屏幕截图。在上课过程中，如果老师发现学生卡住了（或者学生告诉老师自己不能看到老师或无法操作课件），就会向技术支持提交一个**网络状况**工单。

![Alt text](screen_shot.png)


技术支持在看到这个工单后，会通过监课界面以学生视角观看课堂；如果技术支持认为*当时*学生课堂正常，就会关闭工单，并将关闭原因标注为“轻微抖动不影响上课”；否则，技术支持就会打电话给家长协助解决网络问题，并将关闭原因标注为“网络卡顿”。

前端开发告诉你，除了网络状况外，设备型号和性能也会导致操作卡顿；例如早期iPad或者低端Android运行火花课堂时也会卡顿。同时，他们也告诉你技术支持的关闭原因填写有噪音：用户当时是否卡，完全取决于技术支持监课的时机。

现在产研试图降低网络工单率。产品经理提出的一个建议是，**如果在工单提交时可以预测网络工单的关闭原因**，那么就可以在老师提交工单时在后台直
接关闭预测为“轻微抖动不影响上课”的工单，而仅放行“网络卡顿”的工单。

你的同事也为你提取了一些正常课堂的数据，用作训练使用。


## 数据字典

type是你的Y数据：
- 0表示正常课堂
- 1表示产生网络工单且关闭原因为“轻微抖动不影响上课”
- 2表示产生网路工单且关闭原因为“网络卡顿”


火花课堂监控两张网络的状态。一张网络是声网，火花用它来传输音频和视频；另一张网络是game server，火花用它来传输师生间课件控制的数据。

声网的监控数据包括

- duration：通话时长，单位为秒，累计值；重置链接后清零。
- txAudioKBitrate:音频发送码率 (Kbps)，瞬时值
- rxAudioKBitrate:音频接收码率 (Kbps)，瞬时值
- txVideoKBitrate:音频(视频)发送码率 (Kbps)，瞬时值
- rxVideoKBitrate:音频（视频）接收码率 (Kbps)，瞬时值
- cpuTotalUsage:当前系统的 CPU 使用率 (%)
- cpuAppUsage:当前 App 的 CPU 使用率 (%)
- userCount: 当前频道内的用户人数
- sentFrameRate: 不重要
- sentBitrate: 不重要

客户端的game server的监控数据包括

- cpu: 上报数列的最高值
- lag: 客户端与game server的ping值
- fps: 客户端的针率
- memory_free：客户端未使用
- memory_app_used
- memory_inactive:

你的同事已经帮你把工单提交前40次监控和工单提交后40次监控的数据整理好了。X_lead_Y表示性能变量X在工单提交前Y次监控的数据；X_lag_Y表示性能变量X在工单提交后Y次监控的数据。X_lead_1和X_lag_1是最接近工单提交的数据。

此外，当性能数据在日志服务器上缺失时，统一填入-999

In [ ]:
'''
sklearn==0.20.1
xgboost==0.6
tensorflow==1.9.0
'''

In [ ]:
#倒入数据,数据展示
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings("ignore")

#数据解压并放在当前路径
huohua_raw=pd.read_csv('data.csv')
huohua_raw=huohua_raw.fillna(-999)
huohua_raw_delna=huohua_raw[huohua_raw.apply(lambda x: -999 not in x.values,1)].copy().reset_index()
del huohua_raw_delna['index']
huohua_raw.head()

In [ ]:
%matplotlib inline
#数据初探 （未删除含缺失值数据）
print('数据初探 （未删除含缺失值数据）')
print()

print('数据维度（含type）：',huohua_raw.shape)
print()
print('标签分布（未删除含缺失值数据）：')
print(huohua_raw.type.value_counts())
print()
print()

print('数据维度（含type,删除含缺失值数据）：',huohua_raw_delna.shape)
print()
print('标签分布（删除含缺失值数据）：')
print(huohua_raw_delna.type.value_counts())

data_cols=set(list(map(lambda x:'_'.join(x.split('_')[:-2]),huohua_raw.columns[1:])))
data_cols=sorted(list(data_cols))
print('采集点个数(不含type)：',len(data_cols))
print()
print('采集点：')
for col in data_cols:
    print(col)
    


#### 经检查与背景描述中所给出的数据采集点一致
#### 含缺失值数据较少，采用删除缺失值的数据做分析

In [ ]:
#数据整合，将flatten数据和标签转化为时序tensor+label;整合方式为 每个字段lead_40->lead_1; lag_40->lag_1
# (n_sample,80*n_cols+1) -> (n_sample,80 ,n_cols),(n_sample,)
#输入：huohua_raw,cols 输出：temp_tensor(n_sample,80 ,n_cols),label(n_sample,),ranked_cols(n_cols)
def data_agg(flat_df,cols):
    temp=flat_df.copy()
    label=np.array(temp.type)
    del temp['type']
    n_sample,n_cols=temp.shape
    
    #重新排序数据的列
    ranked_cols=[]
    tensor_cols=[]
    #按照时间顺序正向排列
    for i in range(40,0,-1):
        for col in cols:
            ranked_cols.append('_'.join([col,'lead',str(i)]))                    
    for i in range(1,41):
        for col in cols:
            ranked_cols.append('_'.join([col,'lag',str(i)]))
    temp=temp[ranked_cols]
    
    #重整数据为(n_sample,80,16)
    temp_tensor=temp.values.reshape(n_sample,80,-1)
    return(temp_tensor,label,ranked_cols)

huohua_tensor,huohua_label,ranked_cols=data_agg(huohua_raw_delna,data_cols)
print('得到重排后的数据形状',np.shape(huohua_tensor))
n_sample=len(huohua_tensor)
#得到形状为（n_sample*80,16）的数据 huohua_delt，以分析不同维度的数据分布
huohua_delt=pd.DataFrame(np.concatenate(huohua_tensor),columns=data_cols)

In [ ]:
#指标分布、极值分析，以得出数据清洗方案
%matplotlib inline

temp=huohua_delt.copy()
label_delt=[]
for l in huohua_label:
    label_delt.extend(np.repeat(l,80))
temp['label']=label_delt

#画出提琴图，分析数据各维度在标签取值不同时分布，观察极值、不合理值
for i in range(len(data_cols)):
    plt.figure(figsize=[10,5])
    sns.violinplot(x=temp.label,y=temp[data_cols[i]])
    plt.title(data_cols[i])
    

#### CPU存在部分值小于0，不符合常识，将其用cpu均值代替;暂不处理极值点，以免对模型造成影响

In [ ]:
##数据清洗，CPU小于0的值用cpu大于0的数的均值替代
cpu_mean=huohua_delt.cpu[huohua_delt.cpu>=0].mean()
huohua_delt.loc[huohua_delt.cpu<0,'cpu']=cpu_mean

huohua_tensor=huohua_delt.values.reshape(-1,80,16)

#存储清洗后数据用于建模
with open('huohua_tensor.pkl','wb') as pkl:
    pickle.dump(file=pkl,obj=huohua_tensor)

In [ ]:
%%time
#特征工程part1: 包含seglearn包中的特征构建函数构建的时间序列特征值、手工特征值（均值、分位点、方差、工单时点前后变化、相关矩阵等）

import seglearn
from seglearn.feature_functions import all_features
from seglearn.transform import FeatureRep
features=all_features()
features.pop('hmean')
features.pop('hist4')
features.pop('corr')

if 'featspace' in dir():
    del featspace
for feat in features.keys():
    if 'featspace' not in dir():
        featspace=pd.DataFrame(features[feat](huohua_tensor),columns=list(map(lambda x:x+'_'+feat,data_cols)))
    else:
        featspace=pd.concat([featspace,
                             pd.DataFrame(features[feat](huohua_tensor),
                                          columns=list(map(lambda x:x+'_'+feat,data_cols)))],
                            axis=1)
#剔除方差为0的特征
for col in featspace.columns:
    if featspace[col].var()==0.:
        del featspace[col]

#手工特征
feat_95=np.percentile(huohua_tensor,95,axis=1)
feat_05=np.percentile(huohua_tensor,5,axis=1)
feat_mean_change=np.mean(huohua_tensor[:,40:,:],axis=1)-np.mean(huohua_tensor[:,:40,:],axis=1) #序列均值前后差异
feat_var_change=np.var(huohua_tensor[:,40:,:],axis=1)-np.var(huohua_tensor[:,:40,:],axis=1)   #序列方差前后差异
feat_95_change=np.percentile(huohua_tensor[:,40:,:],95,axis=1)\
               -np.percentile(huohua_tensor[:,:40,:],95,axis=1)   #序列95分位点前后差异
feat_05_change=np.percentile(huohua_tensor[:,40:,:],5,axis=1)\
               -np.percentile(huohua_tensor[:,:40,:],5,axis=1)   #序列5分位点前后差异
feat_median_change=np.median(huohua_tensor[:,40:,:],axis=1)-np.median(huohua_tensor[:,:40,:],axis=1)   #序列中位数前后差异

#交叉相关性特征 a与b的相关性column_name为a&b
if 'feat_coef' in dir():
    del feat_coef
for i in range(16):
    for j in range(i+1,16):
        temp_coef=list(map(lambda x:np.corrcoef(x[:,0],x[:,1])[0,1],huohua_tensor[:,:,[i,j]]))
        if 'feat_coef' not in dir():
            feat_coef=pd.DataFrame(temp_coef,columns=[data_cols[i]+"&"+data_cols[j]])
        else:
            temp_coef=pd.DataFrame(temp_coef,columns=[data_cols[i]+"&"+data_cols[j]])
            feat_coef=pd.concat([feat_coef,temp_coef],axis=1)

#将特征转化为dataframe并命名表头
feat_mat=feat_coef.copy()
feat_95=pd.DataFrame(feat_95,columns=map(lambda x:x+'_95',data_cols))
feat_05=pd.DataFrame(feat_05,columns=map(lambda x:x+'_05',data_cols))
feat_mean_change=pd.DataFrame(feat_mean_change,columns=map(lambda x:x+'_mean_change',data_cols))
feat_var_change=pd.DataFrame(feat_var_change,columns=map(lambda x:x+'_var_change',data_cols))
feat_95_change=pd.DataFrame(feat_95_change,columns=map(lambda x:x+'_95_change',data_cols))
feat_05_change=pd.DataFrame(feat_05_change,columns=map(lambda x:x+'_05_change',data_cols))
feat_median_change=pd.DataFrame(feat_median_change,columns=map(lambda x:x+'_median_change',data_cols))

#part1总特征矩阵
feat_mat=pd.concat([feat_mat,
                    featspace,
                    feat_95,
                    feat_05,
                    feat_mean_change,
                    feat_var_change,
                    feat_95_change,
                    feat_05_change,
                    feat_median_change],axis=1)
print('part1特征维度:',feat_mat.shape[1])

In [ ]:
%%time
#特征工程part2: tsfresh 时序特征抽取：包含变换、峰度、时序复杂程度等统计特征
import tsfresh as tsf 
from tsfresh.feature_extraction.feature_calculators import *

#tsfresh特征抽取函数只能应用于pandas.Series，此函数将函数拓展到三维tensor
#tensor:三维张量(?,time_steps,n_cols)，待特征抽取的时序数据
#tsfresh_func:特征抽取函数，作用与pandas.Series
#func_name:函数名称，用于标记生成特征的表头
#returned_list:函数是否返回list，若是则返回list的第一个数值对象
def tsfresh_feat(tensor,cols,tsfresh_func,func_name,returned_list=False):
    for i in range(len(cols)):
        temp=tensor[:,:,i].reshape(len(tensor),-1)
        if not returned_list:
            feat_temp=list(map(lambda x:tsfresh_func(x),temp))
        else:
            feat_temp=list(map(lambda x:list(tsfresh_func(x))[0][1],temp))
        if 'mat' not in dir():
            mat=pd.DataFrame(feat_temp,columns=[cols[i]+'_'+func_name])
        else:
            mat=pd.concat([mat,pd.DataFrame(feat_temp,columns=[cols[i]+'_'+func_name])],axis=1)
    return(mat)

#绝对傅里叶变换的谱统计量
param = [{'coeff': 2, 'attr': 'angle'}]
fft_coef_df=tsfresh_feat(tensor=huohua_tensor,
                      cols=data_cols,
                      tsfresh_func=lambda x:fft_coefficient(x,param),
                      func_name='fft_coef',
                      returned_list=True)

#一阶差分绝对和
absolute_sum_of_changes_df=tsfresh_feat(tensor=huohua_tensor,
                                     cols=data_cols,
                                     tsfresh_func=absolute_sum_of_changes,
                                     func_name='absolute_sum_of_changes')
#各阶自相关系数的聚合统计特征
param = [{'f_agg': 'mean', 'maxlag':2}]
agg_autocorrelation_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:agg_autocorrelation(x,param),
                                 func_name='agg_autocorrelation',
                                 returned_list=True)
#基于分块时序聚合值的线性回归
param = [{'f_agg': 'mean','attr': 'pvalue', 'chunk_len': 2}]
agg_linear_trend_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:agg_linear_trend(x,param),
                                 func_name='agg_linear_trend',
                                 returned_list=True)
#近似熵
approximate_entropy_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:approximate_entropy(x,m=10,r=0.1),
                                 func_name='approximate_entropy')

#自回归系数
param = [{'coeff': 0, 'k': 10}]
ar_coefficient_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:ar_coefficient(x,param),
                                 func_name='ar_coefficient',
                                 returned_list=True)

#峰度
kurtosis_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=kurtosis,
                                 func_name='kurtosis')

#最大值相对位置
last_location_of_maximum_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=last_location_of_maximum,
                                 func_name='last_location_of_maximum')
last_location_of_maximum_df=(last_location_of_maximum_df-0.5).applymap(abs)

#最小值最近位置
last_location_of_minimum_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=last_location_of_minimum,
                                 func_name='last_location_of_minimum')
last_location_of_minimum_df=(last_location_of_minimum_df-0.5).applymap(abs)

#线性回归系数
param = [{'attr': 'pvalue'}]
linear_trend_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:linear_trend(x,param),
                                 func_name='linear_trend',
                                 returned_list=True)
#均值上的最长连续子列长度
longest_strike_above_mean_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=longest_strike_above_mean,
                                 func_name='longest_strike_above_mean')
#均值下的最长连续子列长度
longest_strike_below_mean_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=longest_strike_below_mean,
                                 func_name='longest_strike_below_mean')
#分组熵
binned_entropy_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:binned_entropy(x,10),
                                 func_name='binned_entropy_10')

#时序数据非线性度量
c3_2_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:c3(x, 2),
                                 func_name='c3_2')

#复杂度，用来评估时间序列的复杂度，越复杂的序列有越多的谷峰。 
cid_ce_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:cid_ce(x, True),
                                 func_name='cid_ce')

#平均值上个数 
count_above_mean_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=count_above_mean,
                                 func_name='count_above_mean')


#小波分析系数 
param = [ {'widths':tuple([2,2,2]), 'coeff': 2, 'w': 2}]
cwt_coefficients_df=tsfresh_feat(tensor=huohua_tensor,
                                 cols=data_cols,
                                 tsfresh_func=lambda x:cwt_coefficients(x,param),
                                 func_name='cwt_coefficients',
                                 returned_list=True)

#part2总特征矩阵
tsfresh_feat=pd.concat([fft_coef_df,
                        absolute_sum_of_changes_df,
                        agg_autocorrelation_df,
                        agg_linear_trend_df,
                        approximate_entropy_df,
                        ar_coefficient_df,
                        kurtosis_df,
                        last_location_of_maximum_df,
                        last_location_of_minimum_df,
                        linear_trend_df,
                        longest_strike_above_mean_df,
                        longest_strike_below_mean_df,
                        binned_entropy_df,
                        c3_2_df,
                        cid_ce_df,
                        count_above_mean_df,
                        cwt_coefficients_df],axis=1)

print('part2特征维度：',tsfresh_feat.shape[1])
feat_mat=pd.concat([feat_mat,tsfresh_feat],axis=1)
print('总特征维度：',feat_mat.shape[1])
feat_mat.to_csv('feat_mat_huohua_cleaned.csv',index=False)

In [ ]:
#读入构造好的特征矩阵
import pandas as pd
import numpy as np
feat_mat=pd.read_csv('feat_mat_huohua_cleaned.csv')
print('总特征维度：',feat_mat.shape[1])


In [ ]:
%matplotlib inline
#特征分析和选择
#当前建模数据量太少（type为1或2的总共1016），使用过多特征建模容易造成过拟合，所以由显著性选择建模特征，以避免维数灾难
from sklearn.feature_selection import RFE,f_classif,chi2
import matplotlib.pyplot as plt
import seaborn as sns


#特征筛选函数，根据特征与label相关性绝对值排序，筛选出大于阈值的特征，特征间相关性大的剔除两者中与label相关性小的
#feat_matrix:特征矩阵
#label:标签
#label_cor_thr: 标签相关性阈值，特征与label相关性高于此值的特征进入待选集合
#feat_cor_thr:  特征相关性阈值，特征间相关性大与此值的剔除两者中与label相关性小的
def feat_select_corr(feat_matrix,label,label_cor_thr,feat_cor_thr):
    temp=feat_matrix.copy()
    temp['label']=label
    label_cor=temp.corr()['label'].abs().sort_values(ascending=False)[1:]
    label_cor=label_cor[label_cor>label_cor_thr]
    feat_cols=list(label_cor.index)
    feat_cor_mat=feat_matrix.corr()
    del_cols=[]
    for i in range(len(label_cor)-1):
        if feat_cols[i] not in del_cols:
            for j in range(i+1,len(label_cor)):
                if abs(feat_cor_mat[feat_cols[i]][feat_cols[j]])>feat_cor_thr:
                    del_cols.append(feat_cols[j])
    
    feat_cols=list(set(feat_cols)-set(del_cols))
    
    print(temp.corr()['label'][feat_cols])
    plt.figure(figsize=[8,6])
    sns.heatmap(feat_mat[feat_cols].corr())
    return(temp[feat_cols])

#特征筛选函数，根据 label-特征 的p-value排序并去除p-value大于阈值的特征，特征间相关性大的特征对中剔除两者中p-value小的
#feat_matrix:特征矩阵
#label:标签
#pval_thr: p-value阈值，特征与label的p-value值低于此值的特征进入待选集合
#feat_cor_thr:  特征相关性阈值，特征间相关性大与此值的剔除两者中与label的p-value较高的
def feat_select_signi(feat_matrix,label,pval_thr,feat_cor_thr):
    temp=feat_matrix.copy()
    temp=temp.fillna(0)
    signi=f_classif(X=np.abs(temp),y=label)[1]
    signi=pd.Series(signi,index=temp.columns).sort_values(ascending=True)
    signi=signi[signi<pval_thr]
    feat_cols=list(signi.index)
    feat_cor_mat=feat_matrix.corr()
    del_cols=[]
    for i in range(len(signi)-1):
        if feat_cols[i] not in del_cols:
            for j in range(i+1,len(signi)):
                if abs(feat_cor_mat[feat_cols[i]][feat_cols[j]])>feat_cor_thr:
                    del_cols.append(feat_cols[j])
    feat_cols=list(set(feat_cols)-set(del_cols))
    
    print(signi[feat_cols])
    plt.figure(figsize=[8,6])
    sns.heatmap(feat_mat[feat_cols].corr())
    plt.title('Feature Covariance Matrix')
    return(temp[feat_cols])

#经验证由显著性筛选出的特征训练的模型稳定性较高，不采用标签相关性标准筛选
#model_mat_cor=feat_select_corr(feat_matrix=feat_mat[huohua_label!=0],
#                           label=huohua_label[huohua_label!=0],
#                           label_cor_thr=0.01,
#                           feat_cor_thr=0.3)

#所以由显著性选择建模特征
feat_select_signi=feat_select_signi(feat_matrix=feat_mat[huohua_label!=0],
                           label=huohua_label[huohua_label!=0],
                           pval_thr=0.04,
                           feat_cor_thr=0.4)

print('用于建模的特征维度：',feat_select_signi.shape[1])

In [ ]:
%%time
#构建简单模型（逻辑回归）评估特征分类效果，将效果提升到一个较高、较稳定的水平后构建复杂模型

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,roc_auc_score,roc_curve
from sklearn.preprocessing import scale

#特征标准化
model_mat=feat_select_signi.copy()
X=pd.DataFrame(scale(model_mat),columns=model_mat.columns)

#由于问题是在工单提出时预测工单是否是网络卡顿造成的（标签为2），所以需要剔除标签为0的样本，只保留提出工单的样本（即标签为1或2）
y=huohua_label[huohua_label!=0]

model_lr=LogisticRegression(penalty='l1')
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42)
model_lr.fit(Xtrain,ytrain)
print('Training AUC value:',roc_auc_score(model_lr.predict(Xtrain),ytrain))
print('Testing  AUC value:',roc_auc_score(model_lr.predict(Xtest),ytest))


train_roc=roc_curve(y_score=model_lr.predict_proba(Xtrain)[:,1],y_true=ytrain,pos_label=2)
test_roc=roc_curve(y_score=model_lr.predict_proba(Xtest)[:,1],y_true=ytest,pos_label=2)

plt.figure(figsize=[8,7])
sns.lineplot(x=train_roc[0],y=train_roc[1],)
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')
plt.title('Training set ROC')
plt.figure(figsize=[8,7])
sns.lineplot(x=test_roc[0],y=test_roc[1])
plt.title('Ttesting set ROC')
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')

In [ ]:
#基于上述特征构建xgboost分类模型
import xgboost as xgb 
from sklearn.metrics import f1_score,precision_recall_curve

#xgb
dtrain=xgb.DMatrix(Xtrain, label=ytrain-1)
dtest=xgb.DMatrix(Xtest, label=ytest-1)

#训练参数
param = {'learning_rate' : 0.001,'max_depth': 3, 
        'min_child_weight': 2, 'gamma': 0.5, 'subsample': 0.8, 'colsample_bytree': 0.6,
        'scale_pos_weight': 1, 'eta': 0.5, 'silent': 1, 'objective': 'binary:logistic'}

param['eval_metric'] = "auc"
plst = param.items()
evallist = [(dtest, 'eval'), (dtrain, 'train')]

#训练
bst=xgb.train(plst, dtrain, 41, evallist)


#模型ROC曲线和AUC值
train_roc=roc_curve(y_score=bst.predict(dtrain),y_true=ytrain-1)
test_roc=roc_curve(y_score=bst.predict(dtest),y_true=ytest-1)

plt.figure(figsize=[8,7])
sns.lineplot(x=train_roc[0],y=train_roc[1])
plt.title('Training set ROC')
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')
plt.figure(figsize=[8,7])
sns.lineplot(x=test_roc[0],y=test_roc[1])
plt.title('Testing set ROC')
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')


#计算不同阈值下的准确率，召回率和f1-score
Dmat1=xgb.DMatrix(data=X,label=huohua_label[huohua_label!=0])
label=huohua_label[huohua_label!=0]
pred=bst.predict(Dmat1)
precision,recall,threshold=precision_recall_curve(pos_label=2,probas_pred=pred,y_true=label)
f1score=((2*p*r)/(p+r) for p,r in zip(precision,recall))
res_frame=pd.DataFrame(list(zip(threshold,precision,recall,f1score)),columns=['threshold','precision','recall','f1score'])
res_frame.plot(kind='line',x='threshold',figsize=[12,6])

#将模型保存
with open('xgb_model1221.pkl','wb') as pkl:
    pickle.dump(file=pkl,obj=bst)

### 模型分析

1. 由于数据维度较高，带标签（1或2）数据量相对较少，在训练模型时引入过多特征会引发模型过拟合，所以根据显著性选择部分特征进行模型训练，在保持模型稳定性的同时达到一定效果,若需要提升模型，需要引入更多带标签数据（标注了1或2的数据），从而应用更多特征训练，并增加如设备型号、客户端版本等数据信息
2. 依据现有模型，建议根据模型预测工单为网络卡顿的概率从大到小排序，工作人员顺序处理工单，可大大提高工作效率
3. 若需要机器自动判断需要工人去处理的工单，建议与业务部门协商，根据对工作量和用户体验的权衡选择切切分点
  （如选择卡顿概率大于0.5035的工单进行处理，处理的工单中约80%都为网络卡顿，所有网络卡顿中覆盖率可达78%）

In [ ]:
#模型特征分析
#画出模型特征权重，并分析导致网络卡顿的原因比重，并筛选出对网络卡顿影响较大的因素
xgboost_feat_imortance=bst.get_fscore()
xgboost_feat_imortance=pd.DataFrame([(k,xgboost_feat_imortance[k]) for k in xgboost_feat_imortance.keys()],\
                                    columns=['feat_name','xgboost_feat_imortance'])\
                       .sort_values('xgboost_feat_imortance',ascending=False).reset_index()
del xgboost_feat_imortance['index']
lr_feat_imortance=model_lr.coef_
lr_feat_imortance=pd.DataFrame(list(zip(list(X.columns),model_lr.coef_[0])),
                               columns=['feat_name','lr_feat_imortance'])


plt.rcParams['figure.figsize'] = [30, 8]
fig,axes=plt.subplots(nrows=2,ncols=1,sharex=True)
feat_importance=pd.merge(xgboost_feat_imortance,lr_feat_imortance,how='left',on='feat_name')
sns.barplot(data=feat_importance,x='feat_name',y='xgboost_feat_imortance',ax=axes[0])#['xgboost_feat_imortance','lr_feat_imortance'])
plt.xlabel(None)

sns.barplot(data=feat_importance,x='feat_name',y='lr_feat_imortance',ax=axes[1])#['xgboost_feat_imortance','lr_feat_imortance'])
plt.xticks(rotation=90)
plt.xlabel(None)

feat_importance[feat_importance.apply(lambda x:(x.xgboost_feat_imortance>=5) \
                            and (abs(x.lr_feat_imortance)>0.05),1)]

### 模型特征分析

根据xgboost模型特征强度和logistic回归模型权重系数大小，可得知部分显著关系：
 1. ping值总和与网络卡顿关联最大，且是正相关
 2. CPU peak出现的相对位置、用户数量连续均值上长度与网络卡顿负相关
 3. CPU总使用量与延迟的相关性越低，说明越可能是网络卡顿造成的学生卡住
 
**可根据模型对特征的分析结果与业务、网络专家沟通，分析是否由较少学生卡住现象的可能，也可与网络专家或搜查更多视频直播相关资料，引入专家知识、规则作为模型的输入特征，以提高模型的准确率和稳定性**


## 利用深度学习解决时间序列分类问题

传统机器学习需要特征构建、特征分析和选择等过程，涉及大量领域知识、特征构建经验知识和工作量，在此探索利用深度学习LSTM神经网络做时间序列分类，将每一个时间序列样本编码成一个向量（LSTM神经网络的最后一个时间的输出）,再利用该向量建立分类预测

In [ ]:
#LSTM for time series classification
import tensorflow as tf
from scipy import stats
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.preprocessing import LabelBinarizer,scale
from sklearn.metrics import roc_auc_score

model_tensor_all=scale(huohua_raw_delna,axis=0)[:,1:].reshape(-1,80,16)
train_test_split_all = np.random.rand(len(model_tensor_all)) < 0.80
model_label_all=LabelBinarizer().fit_transform(huohua_label)
train_x_all = model_tensor_all[train_test_split_all]
train_y_all = model_label_all[train_test_split_all]
test_x_all = model_tensor_all[~train_test_split_all]
test_y_all = model_label_all[~train_test_split_all]

model_tensor=scale(huohua_raw_delna[huohua_label!=0],axis=0)[:,1:].reshape(-1,80,16)
train_test_split = np.random.rand(len(model_tensor)) < 0.80
model_label=LabelBinarizer().fit_transform(huohua_label)[huohua_label!=0,:]
train_x = model_tensor[train_test_split]
train_y = model_label[train_test_split]
test_x = model_tensor[~train_test_split]
test_y = model_label[~train_test_split]

tf.reset_default_graph()

learning_rate = 0.0001
batch_size = 50
total_batches_all = (train_x_all.shape[0]//batch_size)
total_batches = (train_x.shape[0]//batch_size)

n_input = 16
n_steps = 80
n_hidden = 64
n_classes = 3

alpha = 0.5

In [ ]:
#模型构建
#因为训练数据较少，模型采用单层lstm神经网络

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cell = rnn_cell.LSTMCell(n_hidden,state_is_tuple = True)
    output, state = tf.nn.dynamic_rnn(cell, x, dtype = tf.float32)
    output_flattened = tf.reshape(output, [-1, n_hidden])
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    output_all = tf.nn.sigmoid(output_logits)
    output_reshaped = tf.reshape(output_all,[-1,n_steps,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), n_steps - 1) 
    
    return output_last, output_all

tf.reset_default_graph()

x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
y_steps = tf.placeholder("float", [None, n_classes])

weight = weight_variable([n_hidden,n_classes])
bias = bias_variable([n_classes])
y_last, y_all = LSTM(x,weight,bias)

all_steps_cost = -tf.reduce_mean(((y_steps[0] * tf.log(y_all))[0]  + (1 - y_steps)[0] * tf.log(1 - y_all)[0])*0.1
                                +((y_steps[1] * tf.log(y_all))[1]  + (1 - y_steps)[1] * tf.log(1 - y_all)[1])*0.4
                                +((y_steps[2] * tf.log(y_all))[1]  + (1 - y_steps)[2] * tf.log(1 - y_all)[2])*0.5
                                )
last_step_cost = -tf.reduce_mean(((y[0] * tf.log(y_last))[0] + ((1 - y)[0] * tf.log(1 - y_last))[0])*0.1
                                +((y[1] * tf.log(y_last))[1] + ((1 - y)[1] * tf.log(1 - y_last))[1])*0.4
                                +((y[2] * tf.log(y_last))[2] + ((1 - y)[2] * tf.log(1 - y_last))[2])*0.5
                                )

loss_function = (alpha * all_steps_cost) + ((1 - alpha) * last_step_cost)

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)


In [ ]:
#利用全量数据训练，并打印每10步训练损失函数

with tf.Session() as session:
    
    tf.global_variables_initializer().run()
    
    #在全量数据上训练150个epoch，以初始化训练lstm网络的遗忘门、输入门、输出门矩阵
    for epoch in range(300):#range(training_epochs):
        for b in range(total_batches_all):    
            offset = (b * batch_size) % (train_y_all.shape[0] - batch_size)
            batch_x_all = train_x_all[offset:(offset + batch_size), :]
            batch_y_all = train_y_all[offset:(offset + batch_size), :]
            batch_y_steps = np.tile(batch_y_all,((train_x_all.shape[1]),1))
            _, c = session.run([optimizer, loss_function],feed_dict={x: batch_x_all, y : batch_y_all, y_steps: batch_y_steps})   
        pred_y = session.run(y_last,feed_dict={x:test_x_all})
        pred_y_train = session.run(y_last,feed_dict={x:train_x_all})
        if (epoch+1)%10==0:
            print('epoch',epoch,"ROC AUCScore-All Labels Train:",
                  roc_auc_score(train_y_all,pred_y_train),
                  'Test:',roc_auc_score(test_y_all,pred_y))
            print('epoch',epoch,"ROC AUCScore-Label=2 Train:",
                  roc_auc_score(train_y_all[train_y_all[:,0]!=1,2],pred_y_train[train_y_all[:,0]!=1,2]),
                  'Test:',roc_auc_score(test_y_all[test_y_all[:,0]!=1,2],pred_y[test_y_all[:,0]!=1,2]))
    
            

### 神经网络模型分析

由于带标签数据量较少，参数量过大（16x64x3 +63x3+3=3267），导致模型无法很好的收敛、泛化，若有更多带标签数据可考虑继续探索深度学习模型的可行性。